# Voxelfilter (Version 1, für Abbildungen verwendet)
Voxelfilter nach Oude Elberink et al. 2013 (und auch Oude Elberink und Khoshelham 2015; Kononen et al. 2024). Hier noch die erste, nicht optimierte Version (s. Abschn. 5.4.6), die aber für die Erzeugung der Abbildungen verwendet wurde. Die Parameter können angepasst werden, um die Auswirkungen bei einzelnen Dateien zu prüfen.

Die Visualisierung des Ergebnisses (mit Open3D) öffnet sich in einem seperaten Fenster. Farben: 
- rot: Kandidaten für Schienenpunkte, geringe Intensität
- orange: Kandidaten für Schienenpunkte, hohe Intensität
- grün: unterer Bereich in Voxeln mit Schienenkandidaten
- blau: Voxel ohne Schienenkandidaten
- hellgrau: fehlender freier Lichtraum
- dunkelgrau: Oberleitung
- schwarz: Voxel mit zu geringer Anzahl an Punkten

In [1]:
import pdal 
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import os
import json
from collections import defaultdict

from interessant import * # Bei Änderungen Kernel neu starten

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Eine interessante Punktwolke kann aus dem Dictionary `interessant` gewählt werden:

In [2]:
keys = list(interessant.keys())
print(keys) 
print(len(keys), "Punktwolken in interessant")

['Einfach', 'Diagonal', 'Weiche', 'Ende', 'Bahnübergang', 'Bahnübergang 2', 'Zug run 14 A', 'Zug run 14 B', 'Zug run 24 A', 'Zug run 24 B', 'Gebäude', 'Gleis hohe Intensität', 'Gleis hohe Intensität 2', 'Feldweg', 'Feld', 'Weiche A', 'Weiche B', 'Zaun', 'Straße', 'Kante in Graben', 'Unterirdischer Bhf', 'Fangschiene Tunnel', 'Fangschiene Tunnel 2', 'Gleis weit abseits', 'Komische Linie', 'Kabeltöpfe', 'Güterzug', 'Güterzug Ende', 'Betondeckel', 'Bahnsteig', 'Bahnsteig Ende', 'Ding neben Gleis', 'Wände', 'Viele Gleise', 'Anfang Weiche', 'OLA gleiche Höhe wie Gleis', '4 Gleise', 'Y', 'Weiche C', 'Weiche D', 'Für template', 'Rand', 'Extrem viele Punkte run24', 'Viele Gleise 2', 'Kreuzung linker Teil', 'Kreuzung rechter Teil', 'Weiche abseits', 'Drei', 'Gestrüpp', 'Weiche Rumgeeier', 'Nicht befahren 1', 'Nicht befahren 2', 'Nicht befahren 3']
53 Punktwolken in interessant


In [3]:
# Messlauf wählen
run = run24
# run = run14

# Dateiname wählen
key = keys[0] 
filename = interessant[key] # Oder filename = interessant["Einfach"]
print(key, filename)

filename = os.path.join(run, filename)
if not os.path.exists(filename):
    raise FileNotFoundError(filename)

Einfach 4473900_5335875.copc.laz


Datei öffnen, Klassifizierung auf 0 setzen

In [5]:
pipeline = pdal.Pipeline([pdal.Reader(filename)])
pipeline.execute()
points = pipeline.arrays[0]

points['Classification'] = 0 # 0 bedeutet Unclassified

In [6]:
xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()
rgb = np.vstack((points['Red'], points['Green'], points['Blue'])).transpose() / 65535.0

# Offset entfernen (aber gerundet, damit Kachelgrenzen ganze Zahlen bleiben)
offset = xyz.mean(axis=0).round() 
xyz -= offset

## Parameter für Voxelfilter

In [4]:
thresh = 30  # Anzahl der Punkte, die im Lichtraum erlaubt sind
majority_tresh  = 0.7 # Ignoriere Voxel, wenn Anteil der "Kandidatenpunkte" größer ist
ground_percentile = 10 # Bodenniveau (noise / low points können darunter liegen)

voxel_size = 25 / 30 # 1.0 bei Oude E.
print("Voxel size:", voxel_size)

minimum_points = 50 # Mindestanzahl der Punkte pro Voxel
height_tresh = 0.5 # Maximale Höhe über Boden, wo sich die Schiene befinden kann

with_normals = False # Visualisierung mit Schatten in Open3D

Voxel size: 0.8333333333333334


## Für Visualisierung mit Open3D

In [7]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)

Settings (Kameraposition etc.) für die Visualisierung

In [8]:
# Viewsettings können im Open3D-Fenster mit strg + c kopiert und hier einfügt werden

viewsettings = '''
{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 11.999975427985191, 11.99998692702502, 13.124079998226534 ],
			"boundingbox_min" : [ -13.000024572014809, -13.00001307297498, -3.9965200017734333 ],
			"field_of_view" : 60.0,
			"front" : [ -0.20468464372193082, -0.82045900926496551, 0.53380821531742795 ],
			"lookat" : [ -2.1145501200370735, -2.6052610037108783, 1.4494799802055294 ],
			"up" : [ 0.19010212482081987, 0.50164960558000959, 0.84392467398461002 ],
			"zoom" : 0.55999999999999983
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}

'''

viewsettings = json.loads(viewsettings)

front = viewsettings["trajectory"][0]["front"]
lookat = viewsettings["trajectory"][0]["lookat"]
up = viewsettings["trajectory"][0]["up"]
zoom = viewsettings["trajectory"][0]["zoom"]

Für Visualisierung verwendete Klassifizierungscodes und Farben

In [9]:
my_classes = {
    "Unclassified": 0,
    "High Points": 13,   
    "No clearance": 14,
    "Missing Rail": 15,
    "Low Points": 16,
    "Rail": 20,
}

color_map = {
    13: [0.3, 0.3, 0.3],
    14: [0.5, 0.5, 0.5], 
    15: [0, 0, 1],
    16: [0, 1, 0],
    20: [1, 0, 0],
}

## Optionale Visualisierung der Punktwolke nach RGB oder Intensity

Mit dem folgenden Codeblock kann die Punktwolke nach RGB oder Intensität visualisiert werden, dazu in entsprechenden Zeilen `#` entfernen

In [10]:
intensity = points['Intensity'] 
intensity_normalized = (intensity - intensity.min()) / (intensity.max() - intensity.min())
colormap = plt.get_cmap("viridis")
intensity_colors = colormap(intensity_normalized)
intensity_colors = intensity_colors[:, :3]



pcd.colors = o3d.utility.Vector3dVector(intensity_colors)
# pcd.colors = o3d.utility.Vector3dVector(rgb)

# o3d.visualization.draw_geometries([pcd], front=front, lookat=lookat, up=up, zoom=zoom) # Visualisierung

## Voxel erzeugen 
Diese Implementierung verwendet Defaultdict, um zu jedem Voxel die dazugehörigen Punkte zu speichern. Die optimierte Version ist ca. 10 % schneller und speichert umgekehrt zu jedem Punkt das dazugehörige Voxel. Die Erzeugung der Visualisierung wurde jedoch nicht auf die optimimerte Implementierung übertragen.

In [11]:
maxp = xyz.max(axis=0)
minp = xyz.min(axis=0)
maxp, minp

(array([11.99997543, 11.99998693, 13.12408   ]),
 array([-13.00002457, -13.00001307,  -3.99652   ]))

In [12]:
voxels = xyz.copy()
voxels[:, :2] = ((xyz[:, :2] - minp[:2]) // voxel_size).astype(int)

In [13]:
# Anzahl der Voxel prüfen
np.ceil((maxp[:2] - minp[:2]) / voxel_size).astype(int)

array([30, 30])

In [14]:
voxel_dict = defaultdict(list) # Sammelt z-Werte der Punkte in jedem Voxel
index_dict = defaultdict(list) # Sammelt Indizes der Punkte in jedem Voxel, um später die Klassifizierung zu setzen

# Füllen der Dictionaries
for idx, (point, voxel) in enumerate(zip(xyz, voxels)):
    voxel_key = tuple(voxel[:2])
    voxel_dict[voxel_key].append(point[2])
    index_dict[voxel_key].append(idx)

## Geometrische Regeln prüfen
Setzt auch die Klassifizierungscodes, nach denen die Farben in der Visualisierung gewählt werden

In [15]:

for key, z_values in voxel_dict.items():
    
    # Schwellenwert Anzahl der Punkte
    if len(z_values) < minimum_points:
        continue

    indices = np.array(index_dict[key])
    z_values = np.array(z_values)
    ground_level = np.percentile(z_values, ground_percentile) # 10. Perzentile in Oude E.

    # Freien Lichtraum prüfen
    count = ((z_values > ground_level + 0.5) & (z_values < ground_level + 4.5)).sum()
    if count <= thresh:
        # Punkte max. 0.5 m über Boden, davon 99.5. Quantil für Schienenoberkante 
        mask = (z_values > ground_level) & (z_values < ground_level + height_tresh)
        try:
            candidates_top = np.percentile(z_values[mask], 99.5) 
        except IndexError:
            # Keine Punkte innerhalb des Intervalls
            points['Classification'][indices] = my_classes["Missing Rail"]
            continue

        # Oude Elberink fordern Höhendifferenz > 0.1 m
        # Und markieren nur Punkte 10 cm unterhalb Schienenoberkante als Kandidaten
        # Hier werden 5 mm darüber ebenfalls berücksichtigt
        if candidates_top - ground_level > 0.1:
            mask = (z_values > candidates_top - 0.1) & (z_values < candidates_top + 0.005)

            # Anteil der potentiellen Kandidaten unter allen Punkten des Voxels prüfen
            if mask.sum() < majority_tresh * len(z_values):  
                points['Classification'][indices[mask]] = my_classes["Rail"]
                points['Classification'][indices[~mask]] = my_classes["Low Points"]
            else:
                # Wenn Kandidaten-Anteil größer als Schwellenwert handelt es sich um Hang
                points['Classification'][indices] = my_classes["Missing Rail"]     
        else:
            # Keine Punkte über dem Boden
            points['Classification'][indices] = my_classes["Missing Rail"]

        # Oberleitung
        mask = (z_values >= ground_level + 4.5)
        points['Classification'][indices[mask]] = my_classes["High Points"]


    else:
        # Kein freier Lichtraum
        points['Classification'][indices] = my_classes["No clearance"]


## Visualisierung

In [16]:
class_colors = np.zeros_like(rgb)
for class_value, color in color_map.items():
    class_colors[points['Classification'] == class_value] = color

In [17]:
# Schwellenwert für Intensität
intensity_threshold = 14500

mask = (points["Classification"] == my_classes["Rail"]) & (points["Intensity"] > intensity_threshold)
class_colors[mask] = [0.9, 0.5, 0]

In [18]:
# Ggf. normals für Visualisierung
if with_normals:
    nn_distance = np.mean(pcd.compute_nearest_neighbor_distance())  
    print(nn_distance)  
    radius_normals=nn_distance*10 
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=50), fast_normal_computation=True) 

In [19]:
pcd.colors = o3d.utility.Vector3dVector(class_colors)
o3d.visualization.draw_geometries([pcd], front=front, lookat=lookat, up=up, zoom=zoom)

Ggf. die klassifizierte Punktwolke als LAZ-Datei schreiben

In [20]:
# out_pipeline = pdal.Writer("test.laz").pipeline(points)
# out_pipeline.execute()